# Import packages

In [1]:
import pandas as pd

In [2]:
import re
import unicodedata

from collections import defaultdict

import pickle

#import Levenshtein
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Upload json files

In [3]:
#files= ['0.json', '1.json', '2.json', '3.json', '4.json','42.json', '15693.json']
files = ['sample.json']
# create an empty list to store the DataFrames
dfsList = [pd.read_json(file, orient='records') for file in files]


# combine all DataFrames into a single DataFrame
crossrefDF = pd.concat(dfsList, ignore_index=True)

# Data preparation 

In [4]:
noAuthors = [i for i in range(len(crossrefDF)) if 'author' not in crossrefDF['items'][i]]

Authors = [i for i in range(len(crossrefDF)) if 'author'  in crossrefDF['items'][i]]

## Rows with authors

In [5]:
crossrefAuth = crossrefDF.iloc[Authors].copy()

crossrefAuth.reset_index(inplace= True)
crossrefAuth.drop(columns = ['index'], inplace = True)


In [6]:
crossrefAuth

,items
0,{'URL': 'http://dx.doi.org/10.1504/ijil.2023.1...
1,{'abstract': '<jats:p>Product management is a ...
2,{'URL': 'http://dx.doi.org/10.1021/acsami.2c18...
3,"{'URL': 'http://dx.doi.org/10.1149/1.1393434',..."
4,{'URL': 'http://dx.doi.org/10.1021/acsomega.2c...
...,...
882,"{'URL': 'http://dx.doi.org/10.1021/jm020002p',..."
883,{'URL': 'http://dx.doi.org/10.15278/isms.2022....
884,{'abstract': '<jats:p>In a 21st century classr...
885,{'URL': 'http://dx.doi.org/10.1016/j.jmrt.2022...


## Extract 'DOI', authors --- number of authors

In [7]:
crossrefAuth.loc[:, 'DOI'] = crossrefAuth['items'].apply(lambda x: x['DOI'])
crossrefAuth.loc[:,'authors'] = crossrefAuth['items'].apply(lambda x: x['author'])

numAuthors = [len(crossrefAuth.iloc[i]['authors']) for i in range(len(crossrefAuth))]

crossrefAuth.loc[:,'# authors'] = numAuthors

## Extract 'affiliations' --- number of affiliations

In [8]:
def getAff(k):
   return [crossrefAuth['authors'][k][j]['affiliation'] for j in range(len(crossrefAuth['authors'][k]))]
    
Affiliations = [getAff(k) for k in range(len(crossrefAuth))]

crossrefAuth.loc[:,'affiliations'] = Affiliations

numAffil = [len(Affiliations[i]) for i in range(len(crossrefAuth))]

crossrefAuth.loc[:,'# Affil'] = numAffil

## Clean 'empty' affiliations

In [9]:
possibleEmptyAff = []

for k in range(len(crossrefAuth)):
    if len(crossrefAuth['affiliations'][k][0]) == 0:
        possibleEmptyAff.append(k)

In [10]:
len(possibleEmptyAff)

610

In [11]:
nonEmptyAff = []

for k in possibleEmptyAff:
    for j in range(len(crossrefAuth['affiliations'].iloc[k])):
        if len(crossrefAuth['affiliations'].iloc[k][j]) != 0:
            nonEmptyAff.append(k)
    
FinalEmptyyAff =  [x for x in possibleEmptyAff if x not in nonEmptyAff] 
FinalNonEmptyAff = [x for x in range(len(crossrefAuth)) if x not in FinalEmptyyAff]

# doiDF: crossrefAuth subdataframe with nonpempty affiliation lists

In [12]:
doiDF = crossrefAuth.iloc[FinalNonEmptyAff].copy()
doiDF.reset_index(inplace = True)
doiDF.drop(columns = ['index'], inplace = True)

In [13]:
doiDF

,items,DOI,authors,# authors,affiliations,# Affil
0,{'URL': 'http://dx.doi.org/10.1021/acsami.2c18...,10.1021/acsami.2c18397,"[{'given': 'Guoxian', 'family': 'Zhang', 'sequ...",8,[[{'name': 'Key Laboratory of Special Function...,8
1,{'URL': 'http://dx.doi.org/10.1021/acsomega.2c...,10.1021/acsomega.2c04273,"[{'given': 'Junlong', 'family': 'Han', 'sequen...",7,[[{'name': 'School of Mechanical Engineering a...,7
2,{'abstract': '<jats:p>The article provides a t...,10.54891/2786-7005-2022-1-13,[{'ORCID': 'http://orcid.org/0000-0002-1686-19...,2,[[{'name': 'Communal Institution of Higher Edu...,2
3,{'abstract': '<jats:title>Abstract</jats:title...,10.4049/jimmunol.158.8.3587,"[{'given': 'S J', 'family': 'Gobin', 'sequence...",4,[[{'name': 'Department of Immunohaematology an...,4
4,{'URL': 'http://dx.doi.org/10.1061/(asce)0887-...,10.1061/(asce)0887-3828(2002)16:3(98),"[{'given': 'Norbert J.', 'family': 'Delatte', ...",2,"[[{'name': 'Assistant Professor, Dept. of Civi...",2
...,...,...,...,...,...,...
273,{'abstract': '<jats:p>The authors highlight th...,10.4018/978-1-6684-7593-5.ch044,"[{'given': 'Renuka', 'family': 'Garg', 'sequen...",2,[[{'name': 'Veer Narmad South Gujarat Universi...,2
274,{'abstract': '<jats:title>Abstract</jats:title...,10.4049/jimmunol.159.11.5372,"[{'given': 'C J', 'family': 'Howard', 'sequenc...",6,"[[{'name': 'The Institute for Animal Health, C...",6
275,{'abstract': '<jats:p>: Overcrowding and house...,10.18231/j.jchm.2022.037,"[{'given': 'Ravikant Rambhai', 'family': 'Pate...",4,"[[], [{'name': 'GMERS Medical College, Valsad,...",4
276,{'abstract': '<jats:title>Abstract</jats:title...,10.4049/jimmunol.162.12.6967,"[{'given': 'Chris A.', 'family': 'Benedict', '...",8,"[[{'name': '*Division of Molecular Immunology,...",8


In [14]:
year = [(doiDF['items'].iloc[i]['issued']['date-parts'][0][0]) for i in range(len(doiDF))]

doiDF['year'] = year

        
        
doiDF.head(10)

,items,DOI,authors,# authors,affiliations,# Affil,year
0,{'URL': 'http://dx.doi.org/10.1021/acsami.2c18...,10.1021/acsami.2c18397,"[{'given': 'Guoxian', 'family': 'Zhang', 'sequ...",8,[[{'name': 'Key Laboratory of Special Function...,8,2022
1,{'URL': 'http://dx.doi.org/10.1021/acsomega.2c...,10.1021/acsomega.2c04273,"[{'given': 'Junlong', 'family': 'Han', 'sequen...",7,[[{'name': 'School of Mechanical Engineering a...,7,2022
2,{'abstract': '<jats:p>The article provides a t...,10.54891/2786-7005-2022-1-13,[{'ORCID': 'http://orcid.org/0000-0002-1686-19...,2,[[{'name': 'Communal Institution of Higher Edu...,2,2022
3,{'abstract': '<jats:title>Abstract</jats:title...,10.4049/jimmunol.158.8.3587,"[{'given': 'S J', 'family': 'Gobin', 'sequence...",4,[[{'name': 'Department of Immunohaematology an...,4,1997
4,{'URL': 'http://dx.doi.org/10.1061/(asce)0887-...,10.1061/(asce)0887-3828(2002)16:3(98),"[{'given': 'Norbert J.', 'family': 'Delatte', ...",2,"[[{'name': 'Assistant Professor, Dept. of Civi...",2,2002
5,{'URL': 'http://dx.doi.org/10.15280/jlm.2022.1...,10.15280/jlm.2022.12.3.138,"[{'given': 'Abhishek', 'family': 'Sharma', 'se...",3,[[{'name': 'Department of Paediatric and Neona...,3,2022
6,{'URL': 'http://dx.doi.org/10.25253/99.2022244...,10.25253/99.2022244.11,[{'ORCID': 'http://orcid.org/0000-0002-0585-54...,1,"[[{'name': 'Aksaray University, Türkiye'}]]",1,2022
7,{'abstract': '<jats:title>Abstract</jats:title...,10.4049/jimmunol.159.11.5535,"[{'given': 'F H', 'family': 'Amante', 'sequenc...",4,[[{'name': 'The Cooperative Research Center fo...,4,1997
8,{'URL': 'http://dx.doi.org/10.1080/23322039.20...,10.1080/23322039.2022.2161774,[{'ORCID': 'http://orcid.org/0000-0002-9452-09...,2,[[{'name': 'Pan African University Institute o...,2,2022
9,{'abstract': '<jats:p>Cryptocurrencies show so...,10.22495/rgcv12i4p5,[{'ORCID': 'http://orcid.org/0000-0001-9173-56...,3,"[[{'name': 'Masaryk University'}], [{'name': '...",3,2022


## (still some cleaning: cases with empty brackets [{}])

In [15]:
for k in range(len(doiDF)):
    if len(doiDF['affiliations'][k][0]) != 0 and doiDF['affiliations'][k][0][0] == {}:
        print(k)

In [16]:
emptyBrackets = [k for k in range(len(doiDF)) if len(doiDF['affiliations'][k][0]) != 0 and doiDF['affiliations'][k][0][0] == {}]

In [17]:
doiDF.iloc[emptyBrackets]

,items,DOI,authors,# authors,affiliations,# Affil,year


In [18]:
doiDF.drop(emptyBrackets, inplace = True)

In [19]:
doiDF.reset_index(inplace = True)

In [20]:
doiDF.drop(columns = ['index'], inplace = True)

In [21]:
len(doiDF)

278

# Clean affiliations 

## is_contained(a,b) map : returns true when a is a substring of b 

In [22]:
def is_contained(s, w):
    words = s.split()  # Split the string 's' into a list of words
    for word in words:
        if word not in w:  # If a word from 's' is not found in 'w'
            return False  # Return False immediately
    return True  # If all words from 's' are found in 'w', return True

## 1. "Unique" affiliations --- number of unique affiliations

In [23]:
uniqueAff = []
error_indices =[] # New list to store error indices
for i in range(len(doiDF)):
    try:
        uniqueAff.append(list(set([x[0] for x in [list(d.values()) for d in [item for sublist in doiDF['affiliations'].iloc[i] for item in sublist if sublist !=[{}] and item !={}]]])))
    except TypeError:
        print("Error occurred for i =", i)
        error_indices.append(i)  # Save the index where the error occurred
    #except IndexError:
     #   print("IndexError occurred for i =", i)
      #  error_indices.append(i)  # Save the index where the IndexError occurred


# Print the error indices
print("Error indices:", error_indices)

Error indices: []


In [24]:
doiDF.drop(error_indices, inplace = True)
doiDF.reset_index(inplace = True)
doiDF.drop(columns = ['index'], inplace = True)

In [25]:
doiDF.loc[:,'uniqueAff'] = uniqueAff

numUniqueAff = [len(doiDF['uniqueAff'].iloc[i]) for i in range(len(doiDF))]

doiDF.loc[:,'# uniqueAff'] = numUniqueAff

## 2. Remove stop words 

In [26]:
doiDF.loc[:,'uniqueAff1'] = doiDF['uniqueAff'].apply(lambda x: [s.lower() for s in x])


In [27]:
stopWords = ['from', 'the', 'of', 'at', 'de','for','et','für','des', 'in','as','a','and']

In [28]:
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopWords]
    return ' '.join(filtered_words)


# apply the function to the column  doiDF['uniqueAff'] to create column doiDF.loc[:,'uniqueAff1']

doiDF.loc[:,'uniqueAff1'] = doiDF['uniqueAff'].apply(lambda x: [remove_stop_words(s) for s in x])


## 3. Remove parenthesis 

In [29]:
def remove_parentheses(text):
   return re.sub(r'\([^()]*\)', '', text)

# apply the function to each list element of column doiDF['uniqueAff1'] to remove substrings inside parentheses

doiDF.loc[:,'uniqueAff1'] = doiDF['uniqueAff1'].apply(lambda x: [remove_parentheses(s) for s in x])


## 4. Remove @#$%characters and umlauts

In [30]:
def replace_umlauts(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    replaced_text = ''.join(c for c in normalized_text if not unicodedata.combining(c))
    return replaced_text

affNoSymbols = []

for i in range(len(list(doiDF['uniqueAff1']))):
    L = list(doiDF['uniqueAff1'])[i]
    for j in range(len(L)):
        L[j] = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', L[j])
        L[j] = L[j].replace("  ", " ")
        L[j] = replace_umlauts(L[j])
        
    affNoSymbols.append(L)



In [31]:
affNoSymbols = [[item for item in inner_list if item != "inc"] for inner_list in affNoSymbols]

doiDF['uniqueAff1'] = affNoSymbols

## 5. Check 'sub'-affiliations (affiliations that are contained in other affiliations of the same DOI)

In [32]:
newAff0 = []

for k in range(len(doiDF)):
    
    L2 = []
    for s1 in doiDF['uniqueAff1'].iloc[k]:
        is_substring = False
        for s2 in doiDF['uniqueAff1'].iloc[k]:
            if s1 != s2 and s1 in s2:
                is_substring = True
                break
        if not is_substring:
            L2.append(s1)
    newAff0.append(L2)

In [33]:
newAffList = [list(set(newAff0[k])) for k in range(len(newAff0))]
doiDF['Unique affiliations'] = newAffList

In [34]:
allAffsList = []

for doi in newAffList:
    for aff in doi:
        if aff not in allAffsList:
            allAffsList.append(aff)
        

## 6. Split strings where ',' or ';' appears    | Apply .lower()

In [35]:
def substringsDict(string):
    split_strings = [re.sub(r'^[\s.]+|[\s.]+$', '', s.strip()) for s in re.split(r'[,;]', string)]
    dict_string = {}
    index = 0

    for value in split_strings:
        if value:
            modified_value = re.sub(r'\buniversit\w*', 'universit', value, flags=re.IGNORECASE)
            dict_string[index] = modified_value.lower()
            index += 1

    return dict_string

In [36]:
newAffkomma = []

for aff in allAffsList:
    newAffkomma.append(substringsDict(aff))



In [37]:
for dict in newAffkomma:
    
    if len(dict)>1:
        for i in range(len(dict)-1):
            if is_contained('progr', dict[i]) and is_contained('dep', dict[i+1]):
                del dict[i]
            elif (is_contained('assistant', dict[i]) or is_contained('researcher', dict[i]) or is_contained('phd', dict[i]) or is_contained('student', dict[i]) or is_contained('section', dict[i]) or is_contained('prof', dict[i]) or is_contained('director', dict[i])) and (not is_contained('school', dict[i+1]) or is_contained('univ', dict[i+1]) or is_contained('inst', dict[i+1]) or is_contained('lab', dict[i+1]) or is_contained('fac', dict[i+1])):
                del dict[i]
            elif (is_contained('engineer', dict[i]) or is_contained('progr', dict[i]) or is_contained('unit', dict[i]) or is_contained('lab', dict[i]) or is_contained('dep', dict[i]) or is_contained('inst', dict[i]) or is_contained('hosp', dict[i]) or is_contained('school', dict[i]) or is_contained('fac', dict[i])) and is_contained('univ', dict[i+1]):
                del dict[i]
            elif is_contained('lab', dict[i]) and (is_contained('college', dict[i+1]) or is_contained('inst', dict[i+1]) or is_contained('dep', dict[i+1]) or is_contained('school', dict[i+1])):
                del dict[i]
            elif is_contained('dep', dict[i]) and (is_contained('tech', dict[i+1]) or is_contained('college', dict[i+1]) or is_contained('inst', dict[i+1]) or  is_contained('hosp', dict[i+1]) or  is_contained('school', dict[i+1]) or  is_contained('fac', dict[i+1])):
                del dict[i]
            elif is_contained('inst',dict[i]) and (is_contained('dep', dict[i+1]) or is_contained('acad', dict[i+1]) or is_contained('hosp', dict[i+1]) or is_contained('fac', dict[i+1]) or is_contained('cent', dict[i+1]) or is_contained('div', dict[i+1])):
                del dict[i]
            elif is_contained('hosp',dict[i]) and is_contained('school', dict[i+1]):
                del dict[i]
         #   elif is_contained('hos',dict[i]) and (is_contained('cen', dict[i+1]):
         #       del dict[i+1]

In [38]:
lightAff = []
for dict in newAffkomma:
    lightAff.append(', '.join(list(dict.values())))
    

In [39]:
removeList = ['university','research institute','laboratory' , 'universit','gmbh', 'inc', 'university of', 'research center', 
'university college','national institute of', 'school of medicine', "university school", 'graduate school of', 'graduate school of engineering', 
'institute of tropical medicine', 'institute of virology', 'faculty of medicine','laboratory', 'university park', 'institute of science','Polytechnic University']

city_names = ["Aberdeen", "Abilene", "Akron", "Albany", "Albuquerque", "Alexandria", "Allentown", "Amarillo", "Anaheim", "Anchorage", "Ann Arbor", "Antioch", "Apple Valley", "Appleton", "Arlington", "Arvada", "Asheville", "Athens", "Atlanta", "Atlantic City", "Augusta", "Aurora", "Austin", "Bakersfield", "Baltimore", "Barnstable", "Baton Rouge", "Beaumont", "Bel Air", "Bellevue", "Berkeley", "Bethlehem", "Billings", "Birmingham", "Bloomington", "Boise", "Boise City", "Bonita Springs", "Boston", "Boulder", "Bradenton", "Bremerton", "Bridgeport", "Brighton", "Brownsville", "Bryan", "Buffalo", "Burbank", "Burlington", "Cambridge", "Canton", "Cape Coral", "Carrollton", "Cary", "Cathedral City", "Cedar Rapids", "Champaign", "Chandler", "Charleston", "Charlotte", "Chattanooga", "Chesapeake", "Chicago", "Chula Vista", "Cincinnati", "Clarke County", "Clarksville", "Clearwater", "Cleveland", "College Station", "Colorado Springs", "Columbia", "Columbus", "Concord", "Coral Springs", "Corona", "Corpus Christi", "Costa Mesa", "Dallas", "Daly City", "Danbury", "Davenport", "Davidson County", "Dayton", "Daytona Beach", "Deltona", "Denton", "Denver", "Des Moines", "Detroit", "Downey", "Duluth", "Durham", "El Monte", "El Paso", "Elizabeth", "Elk Grove", "Elkhart", "Erie", "Escondido", "Eugene", "Evansville", "Fairfield", "Fargo", "Fayetteville", "Fitchburg", "Flint", "Fontana", "Fort Collins", "Fort Lauderdale", "Fort Smith", "Fort Walton Beach", "Fort Wayne", "Fort Worth", "Frederick", "Fremont", "Fresno", "Fullerton", "Gainesville", "Garden Grove", "Garland", "Gastonia", "Gilbert", "Glendale", "Grand Prairie", "Grand Rapids", "Grayslake", "Green Bay", "GreenBay", "Greensboro", "Greenville", "Gulfport-Biloxi", "Hagerstown", "Hampton", "Harlingen", "Harrisburg", "Hartford", "Havre de Grace", "Hayward", "Hemet", "Henderson", "Hesperia", "Hialeah", "Hickory", "High Point", "Hollywood", "Honolulu", "Houma", "Houston", "Howell", "Huntington", "Huntington Beach", "Huntsville", "Independence", "Indianapolis", "Inglewood", "Irvine", "Irving", "Jackson", "Jacksonville", "Jefferson", "Jersey City", "Johnson City", "Joliet", "Kailua", "Kalamazoo", "Kaneohe", "Kansas City", "Kennewick", "Kenosha", "Killeen", "Kissimmee", "Knoxville", "Lacey", "Lafayette", "Lake Charles", "Lakeland", "Lakewood", "Lancaster", "Lansing", "Laredo", "Las Cruces", "Las Vegas", "Layton", "Leominster", "Lewisville", "Lexington", "Lincoln", "Little Rock", "Long Beach", "Lorain", "Los Angeles", "Louisville", "Lowell", "Lubbock", "Macon", "Madison", "Manchester", "Marina", "Marysville", "McAllen", "McHenry", "Medford", "Melbourne", "Memphis", "Merced", "Mesa", "Mesquite", "Miami", "Milwaukee", "Minneapolis", "Miramar", "Mission Viejo", "Mobile", "Modesto", "Monroe", "Monterey", "Montgomery", "Moreno Valley", "Murfreesboro", "Murrieta", "Muskegon", "Myrtle Beach", "Naperville", "Naples", "Nashua", "Nashville", "New Bedford", "New Haven", "New London", "New Orleans", "New York", "New York City", "Newark", "Newburgh", "Newport News", "Norfolk", "Normal", "Norman", "North Charleston", "North Las Vegas", "North Port", "Norwalk", "Norwich", "Oakland", "Ocala", "Oceanside", "Odessa", "Ogden", "Oklahoma City", "Olathe", "Olympia", "Omaha", "Ontario", "Orange", "Orem", "Orlando", "Overland Park", "Oxnard", "Palm Bay", "Palm Springs", "Palmdale", "Panama City", "Pasadena", "Paterson", "Pembroke Pines", "Pensacola", "Peoria", "Philadelphia", "Phoenix", "Pittsburgh", "Plano", "Pomona", "Pompano Beach", "Port Arthur", "Port Orange", "Port Saint Lucie", "Port St. Lucie", "Portland", "Portsmouth", "Poughkeepsie", "Providence", "Provo", "Pueblo", "Punta Gorda", "Racine", "Raleigh", "Rancho Cucamonga", "Reading", "Redding", "Reno", "Richland", "Richmond", "Richmond County", "Riverside", "Roanoke", "Rochester", "Rockford", "Roseville", "Round Lake Beach", "Sacramento", "Saginaw", "Saint Louis", "Saint Paul", "Saint Petersburg", "Salem", "Salinas", "Salt Lake City", "San Antonio", "San Bernardino", "San Buenaventura", "San Diego", "San Francisco", "San Jose", "Santa Ana", "Santa Barbara", "Santa Clara", "Santa Clarita", "Santa Cruz", "Santa Maria", "Santa Rosa", "Sarasota", "Savannah", "Scottsdale", "Scranton", "Seaside", "Seattle", "Sebastian", "Shreveport", "Simi Valley", "Sioux City", "Sioux Falls", "South Bend", "South Lyon", "Spartanburg", "Spokane", "Springdale", "Springfield", "St. Louis", "St. Paul", "St. Petersburg", "Stamford", "Sterling Heights", "Stockton", "Sunnyvale", "Syracuse", "Tacoma", "Tallahassee", "Tampa", "Temecula", "Tempe", "Thornton", "Thousand Oaks", "Toledo", "Topeka", "Torrance", "Trenton", "Tucson", "Tulsa", "Tuscaloosa", "Tyler", "Utica", "Vallejo", "Vancouver", "Vero Beach", "Victorville", "Virginia Beach", "Visalia", "Waco", "Warren", "Washington", "Waterbury", "Waterloo", "West Covina", "West Valley City", "Westminster", "Wichita", "Wilmington", "Winston", "Winter Haven", "Worcester", "Yakima", "Yonkers", "York", "Youngstown"]

city_names = [x.lower() for x in city_names]

In [40]:
for dict in newAffkomma:
    for i in list(dict.keys()):

        if dict[i] in city_names+removeList:
            del dict[i]


In [41]:
affDF = pd.DataFrame()
affDF['Original Affiliations'] = allAffsList
affDF['Light Affiliations'] = lightAff
affDF['Keywords'] =  [list(d.values()) for d in newAffkomma]


# Labels based on legalnames of openAIRE's organizations

In [42]:
uniList = ['institu', 'istitut', 'univ', 'coll', 'center','polytechnic', 'centre' , 'cnrs', 'faculty','school' , 'academy' , 'akadem','école', 'hochschule' , 'ecole', 'tech', 'observ']

labList = ['lab']

hosplList = ['hospital' ,'clinic', 'hôpital', 'klinik','oncol','medical']

gmbhList = ['gmbh', 'company' , 'industr', 'etaireia' , 'corporation', 'inc']

musList =  ['museum', 'library']

foundList =  ['foundation' , 'association','organization' ,'society', 'group' ]

deptList = ['district' , 'federation'  , 'government' , 'municipal' , 'county','council', 'agency']
# miistry -> out

unknownList = ['unknown']

#######   Dictionaries ##########

uniDict = {k: 'Univ/Inst' for k in uniList}   

labDict = {k: 'Laboratory' for k in labList} 

hosplDict = {k: 'Hospital' for k in hosplList}   

gmbhDict = {k: 'Company' for k in gmbhList}   

musDict = {k: 'Museum' for k in musList}   

#schoolDict = {k: 'School' for k in schoolList}   

foundDict = {k: 'Foundation' for k in foundList}   

deptDict = {k: 'Government' for k in deptList}   

unknownDict =  {k: 'Unknown' for k in unknownList}   

categDictsList = [uniDict, labDict, hosplDict, gmbhDict, musDict, #schoolDict, 
                  foundDict, deptDict, unknownDict]

################# Final Dictionary #####################

categDicts = {}
i = 0
while i in range(len(categDictsList)):
    categDicts.update(categDictsList[i])
    i = i+1
    
    


## affiliationsDict

In [43]:
affiliationsDict = {}

for i in range(len(affDF)):
    affiliationsDict[i] = affDF['Keywords'].iloc[i]

In [44]:
d_new = {}

# iterate over the keys of affiliationsDict
for k in range(len(affiliationsDict)):
    # get the list associated with the current key in affiliationsDict
    L = affiliationsDict.get(k, [])
    mapped_listx = [[s, v] for s in L for k2, v in categDicts.items() if k2 in s]
    

    # add the mapped list to the new dictionary d_new
    d_new[k] = mapped_listx

In [45]:
affDF['Dictionary'] = list(d_new.values())

In [46]:
notInList = [i for i in range(len(affDF)) if affDF['Dictionary'].iloc[i] == []]
    

In [47]:
len(doiDF)  - len(notInList)

241

In [48]:
len(notInList)

37

# doiDF1 ['DOI', 'affiliations', 'Dictionary', 'uniqueAff2','# authors','# uniqueAff']

## New column: category based on the labels 

In [49]:
category = [', '.join(list(set([x[1] for x in affDF['Dictionary'].iloc[i]]))) for i in range(len(affDF))]
    


In [50]:
affDF.loc[:, 'Category'] = category


### new label: rest

In [51]:
for i in range(len(affDF)):
    if affDF['Category'].iloc[i] == '':
        affDF.iloc[i, affDF.columns.get_loc('Category')] = 'Rest'


In [52]:
affiliationsSimple = [
    list(set([x[0] for x in affDF['Dictionary'].iloc[i]]))
    for i in range(len(affDF))
]


In [53]:
affDF['Keywords'] = affiliationsSimple

# radius

In [54]:
def strRadiusU(string):
    string = string.lower()
    radius = 3
    
    strList = string.split()
    indices = []
    result = []

    for i, x in enumerate(strList):
        if is_contained('univers',x):
            indices.append(i)
            
    for r0 in indices:
        lmin =max(0,r0-radius)
        lmax =min(r0+radius, len(strList))
        s = strList[lmin:lmax]
        
        result.append(' '.join(s))
    
    return result 

In [55]:
def strRadiusH(string):
    string = string.lower()
    radius = 3
    
    strList = string.split()
    indices = []
    result = []

    for i, x in enumerate(strList):
        if is_contained('hospital',x):
            indices.append(i)
            
    for r0 in indices:
        lmin =max(0,r0-radius-1)
        lmax =min(r0+radius, len(strList))
        s = strList[lmin:lmax]
        
        result.append(' '.join(s))
    
    return result 

In [56]:
def strRadiusC(string):
    string = string.lower()
    radius = 2
    
    strList = string.split()
    indices = []
    result = []

    for i, x in enumerate(strList):
        if is_contained('clinic',x) or is_contained('klinik',x):
            indices.append(i)
            
    for r0 in indices:
        lmin =max(0,r0-radius-1)
        lmax =min(r0+radius, len(strList))
        s = strList[lmin:lmax]
        
        result.append(' '.join(s))
    
    return result 

In [57]:
affiliationsSimpleN = []

for i in range(len(affiliationsSimple)):
    inner = []
    for str in affiliationsSimple[i]:
        if 'universit' in str:
            for x in strRadiusU(str):
                inner.append(x)
        elif 'hospital' in str or 'hôpital' in str:
            for x in strRadiusH(str):
                inner.append(x)
        elif 'clinic' in str or 'klinik' in str:
            for x in strRadiusH(str):
                inner.append(x)
                
        else:
            inner.append(str)
    
    affiliationsSimpleN.append(inner)      
            

In [58]:
affDF['Keywords'] = affiliationsSimpleN

# UNIVS & LABS

In [59]:
univLabs = [i for i in range(len(affDF)) if 'Laboratory' in affDF['Category'].iloc[i] 
            or 'Univ/Inst' in  affDF['Category'].iloc[i]]

In [60]:

univLabsDF = affDF.iloc[univLabs].copy()
univLabsDF.reset_index(inplace = True)
univLabsDF.drop(columns = ['index'], inplace = True)

In [61]:
len(univLabsDF)/len(affDF)

0.8715203426124197

# Load files from openAIRE

In [62]:

with open('dixOpenOrgId.pkl', 'rb') as f:
    dixOpenOrgId = pickle.load(f)

## Clean/modify the files

In [63]:
def filter_key(key):
    # Remove all non-alphanumeric characters except Greek letters and Chinese characters
    modified_key = re.sub(r'[^\w\s,Α-Ωα-ωぁ-んァ-ン一-龯，]', '', key)
    modified_key = re.sub(r'\buniversit\w*', 'universit', modified_key, flags=re.IGNORECASE)
    modified_key = modified_key.replace(' and ', ' ')
    return modified_key

    
def filter_dictionary_keys(dictionary):
    filtered_dict = {}
    for key, value in dictionary.items():
        filtered_key = filter_key(key)
        filtered_dict[filtered_key] = value
    return filtered_dict

In [64]:
def cleanDict(dix):
    dix1 =  {k.replace(',', ''): v for k, v in dix.items()}
    
    dix1 = {replace_umlauts(key): value
    for key, value in dix1.items()}
    
    dix1 = filter_dictionary_keys(dix1)
    
    dix2 = {}
    
    for key, value in dix1.items():
        updated_key = ' '.join([word for word in key.split() if word.lower() not in stopWords])
        dix2[updated_key] = value
        
    for x in list(dix2.keys()):
        if len(x) <3:
            del dix2[x]
            
    if 'universit hospital' in list(dix2.keys()):
        del dix2['universit hospital']
        
    if 'universit school' in list(dix2.keys()):
        del dix2['universit school']
        
    if 'ni universit' in list(dix2.keys()):
        del dix2['ni universit']

        
    if 's v universit' in list(dix2.keys()):
        del dix2['s v universit']

    if 'k l universit' in list(dix2.keys()):
        del dix2['k l universit']
        
    return dix2
    

In [65]:
dixOpenOrgId2 = cleanDict(dixOpenOrgId)

In [66]:
def findID(name):
    lnames = []
    for x in list(dixOpenOrgId2.keys()):
        if name.lower() in x:
            lnames.append(x)
    return lnames

# MATCHINGS

## Helper functions

### Clean the matchings

In [67]:
def bestSimScore(l1, l2, l3, l4, simU, simG):
    """
    Finds the best match between a 'key word' and several legal names from the OpenAIRE database.
    ---> corrects special cases in the main map that follows

    Args:
        l1: List of light affiliations.
        l2: number of candidates.
        l3: List of pairs.
        l4: mult

    Returns:
        List: Resulting list containing OpenAIRE names and their similarity scores.
    """
    
    vectorizer = CountVectorizer()
    numUniv = sum([(l1[i].lower()).count('univ') for i in range(len(l1))])
    result = []
    for i in range(len(l1)):
        best = [] 
        s = l1[i]

    
        for j in range(len(l3)):
            x = l3[j][1] 
           
            if [x, l3[j][2]] in result:
                    continue
            
            if l4[l3[j][0]] == 1:
               
                if  is_contained('univ', x.lower()) and  l3[j][2]> simU:
                    result.append([x, l3[j][2]])
                elif  l3[j][2] >simG:
                    result.append([x, l3[j][2]])

                
              
            elif l3[j][2] >=0.99 and (is_contained("univ", x.lower()) or is_contained("college", x.lower()) or  is_contained("center", x.lower()) or  is_contained("schule", x.lower())): # If the similarity score of a pair (s,x) was 1, we store it to results list
                result.append([l3[j][1], 1])
                
            else:
                try:
            #        x_contains_university = is_contained("university", x.lower())
                    if not is_contained("univ", x.lower()):
                        continue  # Skip if x does not contain "university" or "univ"
                    
                    if (is_contained('hosp', x.lower()) and not is_contained('hosp', s)) or (not is_contained('hosp', x.lower()) and is_contained('hosp', s)):
                        continue
                    s_vector = vectorizer.fit_transform([s]).toarray() #Else we compute the similarity of s with the original affiiation name
                    x_vector = vectorizer.transform([x]).toarray()

                    

                    # Compute similarity between the vectors
                    similarity = cosine_similarity(x_vector, s_vector)[0][0]
                    if similarity> 0.1:


                        best.append([x, similarity])#(similarity+similarity2)/2])
                except:
                    KeyError
                    
        if best:
            max_numbers = defaultdict(float)
            for item in best:
                string, number = item
                max_numbers[string] = max(max_numbers[string], number)

# Create a new list with the elements having the maximum number for each string
            reduced_best = [[string, number] for string, number in best if number == max_numbers[string]]


            reduced_best.sort(key=lambda x: x[1], reverse=True)
 
            result = result + reduced_best
                
    univ_list = []
    other_list = []
    
    for r in result:
        if is_contained('univ',r[0]):
            univ_list.append(r)
        else:
            other_list.append(r)
    
    limit =  min(numUniv, l2)

    if len(univ_list)> limit:
        result = univ_list[:limit] + other_list
                
    return result

### Find rows with multiple mathcings

In [68]:
def index_multipleMatchings(df):
    multipleMatchings = []
    mult = []

    for i in range(len(df)):
        result_dict = {}
        

        for t in [t[0] for t in df.Pairs.iloc[i]]:
            key = t
            if key in result_dict:
                result_dict[key] += 1
                multipleMatchings.append(i)
                
            else:
                result_dict[key] = 1
        mult.append(result_dict)
    return [list(set(multipleMatchings)), mult]

## Main map

In [69]:
def Aff_Ids(m, DF, dixOpenAIRE, simU, simG):
    
    """
    Matches affiliations in DataFrame 'DF' with names from dictionary 'dixOpenAIRE' and their openAIRE ids based on similarity scores.

    Args:
        m (int): The number of DOIs to check.
        DF (DataFrame): The input DataFrame containing affiliation data.
        dixOpenAIRE (dict): A dictionary of names from OpenAIRE.
        simU (float): Similarity threshold for universities.
        simG (float): Similarity threshold for non-universities.

    Returns:
        DataFrame: The final DataFrame with matched affiliations and their corresponding similarity scores.
    """
    vectorizer = CountVectorizer()

    lnamelist = list(dixOpenAIRE.keys())
    dix = {}    # will store indeces and legalnames of organizations of the DOI { i : [legalname1, legalname2,...]}
    deiktes = []  # stores indeces where a match is found
    similarity_ab = [] # stores lists of similarity scores of the mathces 
    pairs = [] #  pairs[i] =  [ [s,x,t] ] where (s,x) is a match and t the corresponding similarity score
    
    for k in range(m):
        similar_k = []
        pairs_k = []


        for s in DF['Keywords'].iloc[k]:

            if s in lnamelist:
                deiktes.append(k)
                similarity = 1
                similar_k.append(similarity)
                
                pairs_k.append((s,s,similarity))

                if k not in dix:
                    dix[k] = [s]
                else:
                    dix[k].append(s)
            else:

                for x in lnamelist:
                    
                    if  is_contained(s, x):

                        x_vector = vectorizer.fit_transform([x]).toarray()
                        s_vector = vectorizer.transform([s]).toarray()

                        # Compute similarity between the vectors
                        similarity = cosine_similarity(x_vector, s_vector)[0][0]
                        if similarity > min(simU, simG):
                            if (is_contained('univ', s) and is_contained('univ', x)) and similarity > simU:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))

                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            elif (not is_contained('univ', s) and not is_contained('univ', x)) and similarity > simG:
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))

                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                                    
                    elif is_contained(x, s):
                        if (is_contained('univ', s) and is_contained('univ', x)):

                            if ' and ' in s:
                                list_s = s.split(' and ')
                                
                                if list_s:
                                    for q in list_s:
                                        if is_contained('univ', q):

                                            q_vector = vectorizer.fit_transform([q]).toarray()
                                            x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                            similarity = cosine_similarity(q_vector, x_vector)[0][0]
                                            if similarity > simU:
                                                similar_k.append(similarity)
                                                deiktes.append(k)
                                                pairs_k.append((s,x,similarity))

                                                if k not in dix:
                                                    dix[k] = [x]
                                                else:
                                                    dix[k].append(x)
                            
                            else: 

                                s_vector = vectorizer.fit_transform([s]).toarray()
                                x_vector = vectorizer.transform([x]).toarray()

                                # Compute similarity between the vectors
                                similarity = cosine_similarity(s_vector, x_vector)[0][0]
                                if similarity > simU: #max(0.82,sim):
                                    similar_k.append(similarity)
                                    deiktes.append(k)
                                    pairs_k.append((s,x,similarity))

                                    if k not in dix:
                                        dix[k] = [x]
                                    else:
                                        dix[k].append(x)
                        elif not is_contained('univ', s) and not is_contained('univ', x):
  

                            s_vector = vectorizer.fit_transform([s]).toarray()
                            x_vector = vectorizer.transform([x]).toarray()

                            # Compute similarity between the vectors
                            similarity = cosine_similarity(s_vector, x_vector)[0][0]
                            if similarity > simG: #max(0.82,sim):
                                similar_k.append(similarity)
                                deiktes.append(k)
                                pairs_k.append((s,x,similarity))

                                if k not in dix:
                                    dix[k] = [x]
                                else:
                                    dix[k].append(x)
                            
        similarity_ab.append(similar_k)   
        similarity_ab = [lst for lst in similarity_ab if lst != []]
        pairs.append(pairs_k)
        
 
    
    
## Define the new Dataframe
    
    affIdDF = pd.DataFrame()
    affIdDF['Original affiliations'] = list(DF['Original Affiliations'].iloc[list(set(deiktes))])

    affIdDF['Light affiliations'] = list(DF['Light Affiliations'].iloc[list(set(deiktes))])

    affIdDF['Candidates for matching'] = list(DF['Keywords'].iloc[list(set(deiktes))])


    affIdDF['Matched openAIRE names'] = list(dix.values())
    affIdDF['# Matched orgs'] = [len(list(dix.values())[i]) for i in range(len(list(dix.values())))]
    

    affIdDF['Similarity score'] = similarity_ab

    Pairs = [lst for lst in pairs if lst]
    affIdDF['Pairs'] = Pairs
    affIdDF['mult'] = index_multipleMatchings(affIdDF)[1]




## Correct the matchings
    needCheck = list(set([i for i in range(len(affIdDF)) for k in list(affIdDF['mult'].iloc[i].values()) if k>1]))
    

    ready = [i for i in range(len(affIdDF)) if i not in needCheck]
    
   
    best = [ bestSimScore([affIdDF['Light affiliations'].iloc[i]], len(affIdDF['Candidates for matching'].iloc[i]), affIdDF['Pairs'].iloc[i],affIdDF['mult'].iloc[i], simU, simG) for i in needCheck]
    best_o = []
    best_s = []
    
    for x in best:
        best_o.append([x[i][0]  for i in range(len(x))])
        best_s.append([round(x[i][1],2)  for i in range(len(x))])
    numMathced = [len(best_s[i]) for i in range(len(needCheck))]
    

    
    dfFinal0 = (affIdDF.iloc[ready]).copy()
    dfFinal0['index'] = ready
    
    dfFinal1 = (affIdDF.iloc[needCheck]).copy()
    dfFinal1['index'] = needCheck
    dfFinal1['Matched openAIRE names'] = best_o
    dfFinal1['Similarity score'] = best_s
    dfFinal1['# Matched orgs'] = numMathced
    
    finalDF =  pd.concat([dfFinal0, dfFinal1])
    finalDF.set_index('index', inplace=True)
    finalDF.sort_values('index', ascending=True, inplace = True)
    
    ids = [[dixOpenAIRE[x] for x in v] for v in finalDF['Matched openAIRE names']]
    numIds = [len(x) for x in ids]

    finalDF['IDs'] = ids
    finalDF['# IDs'] = numIds
    finalDF = finalDF[~(finalDF['# Matched orgs'] == 0)]
    
    finalDF = finalDF.reset_index(drop=True)
    perc = 100*len(finalDF)/m



    
    
    return [perc,finalDF, affIdDF, needCheck]
    


In [70]:
result = Aff_Ids(len(univLabsDF), univLabsDF, dixOpenOrgId2, 0.7,0.82)

# Output

# 1. Excel [Affiliations' matchings]

In [71]:
affsMatch = result[1][['Original affiliations','Candidates for matching','Matched openAIRE names','Similarity score']]

In [72]:
affsMatch.to_excel('affilMatch.xlsx', index=False)


# 2. Εxcel [Dois' matchibgs]

In [73]:
dict_aff_open = {x: y for x, y in zip(result[1]['Original affiliations'], result[1]['Matched openAIRE names'])}
dict_aff_id = {x: y for x, y in zip(result[1]['Original affiliations'], result[1]['IDs'])}
dict_aff_score = {x: y for x, y in zip(result[1]['Original affiliations'], result[1]['Similarity score'])}

In [74]:
pids = []
for i in range(len(doiDF)):
    pidsi = []
    for aff in doiDF['Unique affiliations'].iloc[i]:
        if aff in list(dict_aff_id.keys()):
            pidsi = pidsi + dict_aff_id[aff]
        elif 'unmatched organization(s)' not in pidsi:
            pidsi = pidsi + ['unmatched organization(s)']
    pids.append(pidsi)
            

In [75]:
names = []
for i in range(len(doiDF)):
    namesi = []
    for aff in doiDF['Unique affiliations'].iloc[i]:
        if aff in list(dict_aff_open.keys()):
            namesi = namesi +  dict_aff_open[aff]
        elif 'unmatched organization(s)' not in namesi:
            namesi = namesi + ['unmatched organization(s)']
    names.append(namesi)

In [76]:
scores = []
for i in range(len(doiDF)):
    scoresi = []
    for aff in doiDF['Unique affiliations'].iloc[i]:
        if aff in list(dict_aff_score.keys()):
            scoresi = scoresi +  dict_aff_score[aff]
        elif 'unmatched organization(s)' not in scoresi:
            scoresi = scoresi + ['-']
    scores.append(scoresi)

In [77]:
doiDF['Matched openAIRE names'] = names
doiDF['IDs'] = pids
doiDF['Scores'] = scores

In [78]:
unmatched = [i for i in range(len(doiDF)) if doiDF['Matched openAIRE names'].iloc[i] == ['unmatched organization(s)']]
        
matched = [i for i in range(len(doiDF))  if i not in unmatched]


In [79]:
finalDF0 =  doiDF.iloc[matched]
finalDF0.reset_index(inplace = True)

finalDF = finalDF0[['DOI',"Unique affiliations",'Matched openAIRE names','IDs', 'Scores']]

def update_Z(row):
    new_Z = []
    for i in range(len(row['IDs'])):
        entry = {'OpenAIREid': row['IDs'][i], 'Confidence': row['Scores'][i]}
        new_Z.append(entry)
    return new_Z

matching = finalDF.apply(update_Z, axis=1)

finalDF.loc[:,'Matchings'] = matching

/var/folders/jt/118r6k3j6tlg5c0vfx7p74vh0000gn/T/ipykernel_34268/2561224304.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF.loc[:,'Matchings'] = matching


In [80]:
finalDF_short = finalDF[['Unique affiliations','Matched openAIRE names','Scores']]

In [81]:
finalDF_short.to_excel('doisMatch.xlsx', index=False)


# 3. JSON [Final output]

In [82]:

doiDF_output = finalDF[['DOI','Matchings']]

doisMatch = doiDF_output.to_json(orient='records', lines=True)

# Save the JSON to a file
with open('doisMatch.json', 'w') as f:
    f.write(doisMatch)